In [1]:
from tensorflow import keras
import tensorflow_datasets.public_api as tfds
import tensorflow as tf

Создаём датасет с цветочками

In [2]:
# Константы
img_height = img_width = 192

# Разбиваем датасет на тренировочную группу и группу валидации
train_data = tf.keras.utils.image_dataset_from_directory(
    "flowers",  # "new_flowers",
    validation_split=0.1,
    subset="training",
    image_size=(img_height, img_width),
    seed=5432,
    shuffle=True,
)

validation_data = tf.keras.utils.image_dataset_from_directory(
    "flowers",  # "new_flowers",
    validation_split=0.1,
    subset="validation",
    image_size=(img_height, img_width),
    seed=5432,
    shuffle=True,
)

# Убираем лейблы (т.к. у нас задача не распознавать изображения)
train_data = train_data.map(lambda x, y: (x/255.0, x/255.0))
validation_data = validation_data.map(lambda x, y: (x/255.0, x/255.0))

Found 1671 files belonging to 5 classes.
Using 1504 files for training.
Found 1671 files belonging to 5 classes.
Using 167 files for validation.


Модель

In [9]:
from keras.layers import (Input, Dense, Conv2D, Conv2DTranspose, BatchNormalization, UpSampling2D,
                          Dropout, Flatten, Reshape, Lambda, MaxPool2D, Concatenate)
from keras.models import Model
from keras.optimizers import RMSprop, Adam
from keras import backend as K

# Константы
filters = 64
hidden_units = 512

# Энкодер
input_img = Input(shape=(img_height, img_width, 3))
x = UpSampling2D((4, 4))(input_img)

def encode_layer(x):
    x = Conv2D(filters, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")(x)
    x = BatchNormalization()(x)

    x = MaxPool2D((2, 2), padding="same")(x)
    return x

for _ in range(10):
    x = encode_layer(x)

x = Flatten()(x)
encoder_output = Dense(hidden_units)(x)

# Декодер
def decode_layer(x, upsampling=(2, 2)):
    x = Conv2D(filters, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")(x)
    x = BatchNormalization()(x)

    x = UpSampling2D(upsampling)(x)
    return x


def dense_layer(x):
    shape = x.shape
    # x = Flatten()(x)
    x = Dense(shape[2], activation="relu", kernel_initializer="he_normal")(x)
    # x = Reshape((shape[1], shape[2], 1))(x)
    return x

x = Reshape((2, 2, hidden_units//4))(encoder_output)

for _ in range(5):
    x = decode_layer(x)
    x = dense_layer(x)

x = decode_layer(x, (3, 3))
x = dense_layer(x)

x = Conv2D(3, (3, 3), activation="sigmoid", padding="same", kernel_initializer="he_normal")(x)
output_img = Reshape((img_height, img_width, 3))(x)

# Модели
vae = Model(input_img, output_img, name="vae")

vae.compile(
    optimizer=RMSprop(1e-3),
    loss=keras.losses.binary_crossentropy,
    loss_weights=[1000],
)

vae.summary()

Model: "vae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 192, 192, 3)]     0         
                                                                 
 up_sampling2d_21 (UpSamplin  (None, 768, 768, 3)      0         
 g2D)                                                            
                                                                 
 conv2d_87 (Conv2D)          (None, 768, 768, 64)      1792      
                                                                 
 batch_normalization_84 (Bat  (None, 768, 768, 64)     256       
 chNormalization)                                                
                                                                 
 conv2d_88 (Conv2D)          (None, 768, 768, 64)      36928     
                                                                 
 batch_normalization_85 (Bat  (None, 768, 768, 64)     256     

In [10]:
vae.fit(
    train_data,
    batch_size=1,
    epochs=20,
    validation_data=validation_data,
)

Epoch 1/20


ResourceExhaustedError: Graph execution error:

Detected at node 'vae/conv2d_87/Relu' defined at (most recent call last):
    File "C:\ProgramData\miniconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\miniconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\ProgramData\miniconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\ProgramData\miniconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\ProgramData\miniconda3\lib\site-packages\ipykernel\kernelapp.py", line 701, in start
      self.io_loop.start()
    File "C:\ProgramData\miniconda3\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\ProgramData\miniconda3\lib\asyncio\windows_events.py", line 321, in run_forever
      super().run

In [ ]:
# from keras.models import save_model, load_model
# vae.load_weights("123")

import matplotlib.pyplot as plt
from random import randint
import numpy as np

batch_size = 1  # Иначе будет OOM (недостаток памяти в gpu)

# validation_data = train_data

# Конвертируем в список
validation_img = generated_img = []
for img, _ in validation_data:
    validation_img.extend(img.numpy())

generated_images = vae.predict(np.array(validation_img))
num_images = 10

# Отображение сгенерированных изображений
plt.figure(figsize=(18, 5))
for i in range(num_images):
    random_num = randint(0, len(validation_img))
    # Исходное изображение
    plt.subplot(2, num_images, i + 1)
    plt.imshow(validation_img[random_num])
    plt.title("Original")
    plt.axis("off")

    # Сгенерированное изображение
    plt.subplot(2, num_images, i + num_images + 1)
    plt.imshow(generated_images[random_num])
    plt.title("Generated")
    plt.axis("off")

plt.tight_layout()
plt.show()